# Train and test the seq2seq model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time


import tensorflow as tf

import tensorflow.contrib.legacy_seq2seq as seq2seq

from sklearn.model_selection import train_test_split

In [2]:
xo = np.load("iseq_n.npy")
yo = np.load("oseq_n.npy")

In [3]:
x = xo[:,:26,:]
y = yo[:,:26]

In [4]:
nxchars = x.shape[2]
print(nxchars)
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ']
print("#ltokens: ", len(ltokens))

30
#ltokens:  28


In [5]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1
print(x_seq_length, y_seq_length)

26 25


In [6]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [7]:
batch_size = 512
nodes = 256
embed_size = 20

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.float32, (None, x_seq_length, nxchars), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
output_embedding = tf.Variable(tf.random_uniform((len(ltokens)+1, embed_size), -1.0, 1.0), name='dec_embedding')
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=inputs, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(ltokens)+1, activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


Train the graph above:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(y_train[0])

[28  6  7  4 16 23  3 11  9  2  5 27 13 27  1  6  3  5 27 27 27 27 27 27
 27 27]


In [9]:
def save(sess):
    saver = tf.train.Saver(None)

    save_path = saver.save(sess, save_path="seq_mod/model", global_step=None)
    print('model saved at %s' % save_path)

In [10]:
def restore(sess):
    saver = tf.train.Saver(None)
    path = "seq_mod/model"
    saver.restore(sess, save_path=path)
    print('model restored from %s' % path)

In [ ]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    restore(sess)
    epochs = 1000
    for epoch_i in range(epochs):
        start_time = time.time()
        for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
            _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
                feed_dict = {inputs: source_batch,
                 outputs: target_batch[:, :-1],
                 targets: target_batch[:, 1:]})

        accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                          accuracy, time.time() - start_time))

        source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

        dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
        for i in range(y_seq_length):
            batch_logits = sess.run(logits,
                        feed_dict = {inputs: source_batch,
                         outputs: dec_input})
            prediction = batch_logits[:,-1].argmax(axis=-1)
            dec_input = np.hstack([dec_input, prediction[:,None]])

        print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))     
        if epoch_i % 5 == 0:
            save(sess)
    
    save(sess)

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Epoch   0 Loss:  1.205 Accuracy: 0.5196 Epoch duration: 28.502s
Accuracy on test set is:  0.441
model saved at seq_mod/model
Epoch   1 Loss:  1.213 Accuracy: 0.5118 Epoch duration: 27.826s
Accuracy on test set is:  0.431
Epoch   2 Loss:  1.205 Accuracy: 0.5208 Epoch duration: 26.976s
Accuracy on test set is:  0.425
Epoch   3 Loss:  1.205 Accuracy: 0.5148 Epoch duration: 27.908s
Accuracy on test set is:  0.437
Epoch   4 Loss:  1.204 Accuracy: 0.5192 Epoch duration: 26.967s
Accuracy on test set is:  0.379
Epoch   5 Loss:  1.201 Accuracy: 0.5238 Epoch duration: 27.678s
Accuracy on test set is:  0.436
model saved at seq_mod/model
Epoch   6 Loss:  1.207 Accuracy: 0.5195 Epoch duration: 26.093s
Accuracy on test set is:  0.326
Epoch   7 Loss:  1.198 Accuracy: 0.5143 Epoch duration: 29.769s
Accuracy on test set is:  0.418
Epoch   8 Loss:  1.195 Accuracy: 0.5266 Epoch duration: 28.015s
Accuracy on test set

Accuracy on test set is:  0.433
Epoch  80 Loss:  1.194 Accuracy: 0.5196 Epoch duration: 27.483s
Accuracy on test set is:  0.346
model saved at seq_mod/model
Epoch  81 Loss:  1.187 Accuracy: 0.5281 Epoch duration: 21.199s
Accuracy on test set is:  0.422
Epoch  82 Loss:  1.185 Accuracy: 0.5283 Epoch duration: 21.135s
Accuracy on test set is:  0.418
Epoch  83 Loss:  1.179 Accuracy: 0.5308 Epoch duration: 22.125s
Accuracy on test set is:  0.425
Epoch  84 Loss:  1.190 Accuracy: 0.5235 Epoch duration: 23.387s
Accuracy on test set is:  0.422
Epoch  85 Loss:  1.189 Accuracy: 0.5262 Epoch duration: 21.351s
Accuracy on test set is:  0.408
model saved at seq_mod/model
Epoch  86 Loss:  1.181 Accuracy: 0.5291 Epoch duration: 30.608s
Accuracy on test set is:  0.417
Epoch  87 Loss:  1.184 Accuracy: 0.5289 Epoch duration: 38.445s
Accuracy on test set is:  0.401
Epoch  88 Loss:  1.180 Accuracy: 0.5255 Epoch duration: 39.339s
Accuracy on test set is:  0.410
Epoch  89 Loss:  1.177 Accuracy: 0.5241 Epoch 

Accuracy on test set is:  0.384
model saved at seq_mod/model
Epoch 161 Loss:  1.178 Accuracy: 0.5316 Epoch duration: 21.843s
Accuracy on test set is:  0.379
Epoch 162 Loss:  1.171 Accuracy: 0.5311 Epoch duration: 21.990s
Accuracy on test set is:  0.394
Epoch 163 Loss:  1.168 Accuracy: 0.5332 Epoch duration: 20.595s
Accuracy on test set is:  0.395
Epoch 164 Loss:  1.168 Accuracy: 0.5313 Epoch duration: 20.297s
Accuracy on test set is:  0.413
Epoch 165 Loss:  1.156 Accuracy: 0.5408 Epoch duration: 21.441s
Accuracy on test set is:  0.424
model saved at seq_mod/model
Epoch 166 Loss:  1.171 Accuracy: 0.5370 Epoch duration: 20.425s
Accuracy on test set is:  0.389
Epoch 167 Loss:  1.169 Accuracy: 0.5334 Epoch duration: 20.500s
Accuracy on test set is:  0.403
Epoch 168 Loss:  1.171 Accuracy: 0.5305 Epoch duration: 20.516s
Accuracy on test set is:  0.414
Epoch 169 Loss:  1.175 Accuracy: 0.5306 Epoch duration: 20.666s
Accuracy on test set is:  0.430
Epoch 170 Loss:  1.166 Accuracy: 0.5343 Epoch 

Epoch 241 Loss:  1.127 Accuracy: 0.5577 Epoch duration: 19.619s
Accuracy on test set is:  0.432
Epoch 242 Loss:  1.130 Accuracy: 0.5542 Epoch duration: 19.634s
Accuracy on test set is:  0.424
Epoch 243 Loss:  1.125 Accuracy: 0.5563 Epoch duration: 19.689s
Accuracy on test set is:  0.431
Epoch 244 Loss:  1.146 Accuracy: 0.5464 Epoch duration: 19.747s
Accuracy on test set is:  0.440
Epoch 245 Loss:  1.142 Accuracy: 0.5463 Epoch duration: 19.652s
Accuracy on test set is:  0.437
model saved at seq_mod/model
Epoch 246 Loss:  1.128 Accuracy: 0.5521 Epoch duration: 19.671s
Accuracy on test set is:  0.430
Epoch 247 Loss:  1.137 Accuracy: 0.5477 Epoch duration: 19.644s
Accuracy on test set is:  0.417
Epoch 248 Loss:  1.130 Accuracy: 0.5550 Epoch duration: 19.730s
Accuracy on test set is:  0.443
Epoch 249 Loss:  1.128 Accuracy: 0.5503 Epoch duration: 19.663s
Accuracy on test set is:  0.430
Epoch 250 Loss:  1.132 Accuracy: 0.5534 Epoch duration: 19.804s
Accuracy on test set is:  0.424
model saved

Accuracy on test set is:  0.435
Epoch 322 Loss:  1.102 Accuracy: 0.5611 Epoch duration: 19.675s
Accuracy on test set is:  0.481
Epoch 323 Loss:  1.099 Accuracy: 0.5603 Epoch duration: 19.705s
Accuracy on test set is:  0.466
Epoch 324 Loss:  1.104 Accuracy: 0.5625 Epoch duration: 19.666s
Accuracy on test set is:  0.472
Epoch 325 Loss:  1.084 Accuracy: 0.5682 Epoch duration: 19.732s
Accuracy on test set is:  0.442
model saved at seq_mod/model
Epoch 326 Loss:  1.093 Accuracy: 0.5675 Epoch duration: 19.727s
Accuracy on test set is:  0.440
Epoch 327 Loss:  1.104 Accuracy: 0.5571 Epoch duration: 19.649s
Accuracy on test set is:  0.459
Epoch 328 Loss:  1.098 Accuracy: 0.5613 Epoch duration: 19.590s
Accuracy on test set is:  0.446
Epoch 329 Loss:  1.088 Accuracy: 0.5707 Epoch duration: 19.692s
Accuracy on test set is:  0.446
Epoch 330 Loss:  1.102 Accuracy: 0.5580 Epoch duration: 19.762s
Accuracy on test set is:  0.486
model saved at seq_mod/model
Epoch 331 Loss:  1.099 Accuracy: 0.5608 Epoch 

Translate on test set

In [12]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Accuracy on test set is:  0.632


Let's randomly take two from this test set and see what it spits out:

In [21]:
# i = random.randint(0, 511)
print(dec_input[i,:])
print(ltokens)
seq = ""
for c in dec_input[i,1:]:
    c = int(c)
    if c != 28:
        seq += ltokens[c] 
        
print("result:", seq)

seq = ""
for c in target_batch[i,1:]:
    c = int(c)
    
    if c != 28:
        seq += ltokens[c] 

print("Correct:", seq)

[28.  7.  8.  3. 11.  3.  5. 27. 12. 27.  8.  1.  3. 27. 27. 27. 27. 27.
 27. 27. 27. 27. 27. 27. 27. 27.]
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha', '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}', ' ']
result: 783+35 = 813             
Correct: 287+56 = 343             


In [ ]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))
    cc = 0
    fcc = 0
    fc = 0
    fl = 0
    nl = 0
    for i in range(len(dec_input)):
        feq = False
        pseq = ""
        for c in dec_input[i,1:]:
            c = int(c)
            if c != 28:
                pseq += ltokens[c] 

        cseq = ""
        cseql = ""
        for c in target_batch[i,1:]:
            c = int(c)

            if c != 28:
                cseq += ltokens[c] 
                cseql += ltokens[c][0]
                if ltokens[c] == "#frac":
                    fc += 1
                    feq = True

        if pseq == cseq:
            cc += 1
            if feq:
                fcc += 1
        cseql = cseql.rstrip()
        if feq:
            fl += len(cseql)
        else:
            nl += len(cseql)
        
    print("Accuracy %.2f %%" % (cc/len(dec_input)*100))  
    print("Accuracy for fraction equations %.2f %%" % (fcc/fc*100))
    print("Accuracy for simple equations %.2f %%" % ((cc-fcc)/(len(dec_input)-fc)*100))
    print("Average length frac: %.1f" % (fl/fc))
    print("Average length simple: %.1f" % (nl/(len(dec_input)-fc)))